In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
''' Import Libraries '''

import os
from tensorflow.keras.models import load_model
import numpy as np


In [ ]:
''' Load Model '''

curr_path = os.getcwd()
model = load_model(curr_path + '/trained models/Five-classification.h5')

In [ ]:
''' Load Data '''
path = 'data path'

calssificatin_type = {"binary":1,"superclasses":5,"subclasses":23}
classification_name="superclasses"
no_of_classes=calssificatin_type[classification_name]

lead_type={"lead-I":1, "bipolar-limb":3 , "unipolar-limb":3, "limb-leads":6 , "precordial-leads":6,"all-lead":12}
lead_name= "all-lead"
no_of_leads=lead_type[lead_name]

x_test  = np.load(path+'x_test.npy',allow_pickle=True)
y_test  = np.load(path + 'y_test_sub.npy',allow_pickle=True)

x_test  = x_test.transpose(0, 2, 1)

x_test  = x_test.reshape(2198, no_of_leads, 1000, 1)


print("x_test  :", x_test.shape)
print("y_test  :", y_test.shape)
print('Data loaded')


from sklearn.preprocessing import MultiLabelBinarizer

if classification_name!="binary":
    mlb = MultiLabelBinarizer()
    mlb.fit(y_train)
    y_train = mlb.transform(y_train)

    mlb = MultiLabelBinarizer()
    mlb.fit(y_test)
    y_test = mlb.transform(y_test)
    print('Data proocessed')

In [ ]:
"""Evaluate the model"""

from sklearn.metrics import precision_recall_curve, f1_score, roc_auc_score, accuracy_score, auc


def sklearn_metrics(y_true, y_pred):
    y_bin = np.copy(y_pred)
    y_bin[y_bin >= 0.5] = 1
    y_bin[y_bin < 0.5]  = 0

    # Compute area under precision-Recall curve
    auc_sum = 0
    for i in range(no_of_classes):
      precision, recall, thresholds = precision_recall_curve(y_true[:, i], y_pred[:,i])
      auc_sum += auc(recall, precision)

    print("Accuracy        : {:.2f}".format(accuracy_score(y_true.flatten(), y_bin.flatten())* 100))
    print("Macro AUC score : {:.2f}".format(roc_auc_score(y_true, y_pred, average='macro') * 100))
    print('AUPRC           : {:.2f}'.format((auc_sum / 5) * 100))
    print("Micro F1 score  : {:.2f}".format(f1_score(y_true, y_bin, average='micro') * 100))


In [ ]:
y_prob = model.predict(x_test)
print("\nTest")
sklearn_metrics(y_test, y_pred_test)